# Naive Bayes

http://scikit-learn.org/stable/modules/naive_bayes.html

Методы sklearn.naive_bayes - это набор методов, которые основаны на применении теоремы Байеса с "наивным" предположением о условной независимости любой пары признаков $(x_i, x_j)$ при условии выбранного значения целевой переменной $y$. 

Теорема Байеса утверждает, что:

$$P(y \mid x_1, \dots, x_n) = \frac{P(y) P(x_1, \dots x_n \mid y)}
                                    {P(x_1, \dots, x_n)}$$

Используя "наивное" предположение о условной независимости пар признаков

$$P(x_i | y, x_1, \dots, x_{i-1}, x_{i+1}, \dots, x_n) = P(x_i | y),$$

можно преобразовать теорему Байеса к

$$P(y \mid x_1, \dots, x_n) = \frac{P(y) \prod_{i=1}^{n} P(x_i \mid y)}
                                    {P(x_1, \dots, x_n)}$$

Поскольку $P(x_1, \dots, x_n)$ - известные значения признаков для выбранного объекта, мы можем использовать следующее правило:

$$\hat{y} = \arg\max_y P(y) \prod_{i=1}^{n} P(x_i \mid y),$$

In [1]:
import scipy
import numpy as np

Скачиваем данные, в sklearn есть модуль datasets, он предоставляет широкий спектр наборов данных

In [2]:
from sklearn.datasets import load_iris

data = load_iris()

# BernoulliNB

Для оценки вероятности $P(x_i \mid y)$ бинаризуем признаки, а затем рассчитаем её как

$$P(x_i \mid y) = P(x_i \mid y) x_i + (1 - P(x_i \mid y)) (1 - x_i)$$

In [3]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import cross_val_score

cross_val_score(BernoulliNB(), data.data, data.target)

/home/tylorn/envs/python_data_science/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


array([0.33333333, 0.33333333, 0.33333333])

In [4]:
cross_val_score(BernoulliNB(binarize=0.1), data.data, data.target)

/home/tylorn/envs/python_data_science/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


array([0.39215686, 0.35294118, 0.35416667])

In [5]:
cross_val_score(BernoulliNB(binarize=1.), data.data, data.target)

/home/tylorn/envs/python_data_science/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


array([0.66666667, 0.66666667, 0.66666667])

In [6]:
cross_val_score(BernoulliNB(binarize=1., alpha=0.1), data.data, data.target)

/home/tylorn/envs/python_data_science/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


array([0.66666667, 0.66666667, 0.66666667])

В sklearn есть стандартная функция для поиска наилучших значений параметров модели

In [7]:
from sklearn.model_selection import GridSearchCV

res = GridSearchCV(BernoulliNB(), param_grid={
    'binarize': [0.,0.1, 0.5, 1., 2, 10, 100.],
    'alpha': [0.1, 0.5, 1., 2, 10., 100.]
}, cv=3).fit(data.data, data.target)

/home/tylorn/envs/python_data_science/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [8]:
res.best_params_, res.best_score_

({'alpha': 0.1, 'binarize': 2}, 0.82)

In [9]:
cross_val_score(BernoulliNB(binarize=2., alpha=0.1), data.data, data.target).mean()

/home/tylorn/envs/python_data_science/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


0.8206699346405228

Добавим параметры

In [10]:
res = GridSearchCV(BernoulliNB(), param_grid={
    'binarize': [0.,0.1, 0.5, 1., 2, 10, 100.],
    'alpha': [0.1, 0.5, 1., 2, 10., 100.],
    'fit_prior': [False, True]
}, cv=3).fit(data.data, data.target)
res.best_params_, res.best_score_

/home/tylorn/envs/python_data_science/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


({'alpha': 0.1, 'binarize': 2, 'fit_prior': False}, 0.82)

Иногда по сетке перебирать параметры слишком избыточное, поэтому имеет смысл использовать RandomziedSearch

In [11]:
from sklearn.model_selection import RandomizedSearchCV

res = RandomizedSearchCV(BernoulliNB(), param_distributions={
    'binarize': scipy.stats.uniform(0, 10),
    'alpha': scipy.stats.uniform(0, 10),
    'fit_prior': [False, True]
}, cv=3).fit(data.data, data.target)
res.best_params_, res.best_score_

/home/tylorn/envs/python_data_science/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


({'alpha': 2.9971561353459886,
  'binarize': 1.8834053545605278,
  'fit_prior': True},
 0.88)

Первый вариант - локализованный перебор

In [12]:
res = RandomizedSearchCV(BernoulliNB(), param_distributions={
    'binarize': scipy.stats.uniform(1.5, 2.5),
    'alpha': scipy.stats.uniform(0.05, 0.15),
    'fit_prior': [False, True]
}, cv=3).fit(data.data, data.target)
res.best_params_, res.best_score_

/home/tylorn/envs/python_data_science/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


({'alpha': 0.18474394222591228,
  'binarize': 1.9283263782134217,
  'fit_prior': True},
 0.86)

In [13]:
res = RandomizedSearchCV(BernoulliNB(), param_distributions={
    'binarize': scipy.stats.uniform(1.5, 2.5),
    'alpha': scipy.stats.uniform(0.05, 0.15),
    'fit_prior': [False, True]
}, cv=3, random_state=42).fit(data.data, data.target)
res.best_params_, res.best_score_

/home/tylorn/envs/python_data_science/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


({'alpha': 0.07339917805043039,
  'binarize': 1.6452090304204987,
  'fit_prior': True},
 0.92)

Второй вариант - больше диапозоны и больше итераций

In [14]:
res = RandomizedSearchCV(BernoulliNB(), param_distributions={
    'binarize': scipy.stats.uniform(0, 10),
    'alpha': scipy.stats.uniform(0, 10),
    'fit_prior': [False, True]
}, cv=3, random_state=42, n_iter=1000).fit(data.data, data.target)
res.best_params_, res.best_score_

/home/tylorn/envs/python_data_science/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


({'alpha': 6.075448519014383,
  'binarize': 1.7052412368729153,
  'fit_prior': False},
 0.9466666666666667)

# MultinomialNB

MultinomialNB реализует алгоритм наивного Байеса для признаков, распределенных мультиномиально. Хорошо работает для задач опреледения категории текста или детектирования спама. На практике также неплохо работает с признаками после TFIDF.

Распределение параметризовано векторами $\theta_y = (\theta_{y1},\ldots,\theta_{yn})$ для каждого класса $y$, где $n$ - количество признаков, $\theta_{yi}$ - вероятность $P(x_i \mid y)$ появления признака $i$ в объекте класса $y$.

Параметры $\theta_y$ оцениваются с помощью сглаженной версии максимального правдоподобия, то есть относительной частоты встречаемости значений признаков (проще думать об этом, как об встречаемости слов в документах класса $y$):

$$\hat{\theta}_{yi} = \frac{ N_{yi} + \alpha}{N_y + \alpha n}$$

где $N_{yi} = \sum_{x \in T} x_i$ - число раз, когда признак $i$ встречается в объектах класса $y$ среди обучающей выборки $T$,
и $N_{y} = \sum_{i=1}^{n} N_{yi}$ - полное число встреч признака во всех классах $y$.

Сглаживающая константа $\alpha \ge 0$ обрабатывает случаи, когда признак не встречается для класса в обучающей выборке и предотвращает нулевые вероятности в дальнейших вычислениях.

Оценим качество MultinomialNB на той же задаче

In [15]:
from sklearn.naive_bayes import MultinomialNB

cross_val_score(MultinomialNB(), data.data, data.target)

/home/tylorn/envs/python_data_science/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


array([1.        , 0.88235294, 1.        ])

Подберём оптимальные параметры

In [16]:
%%time
res = GridSearchCV(MultinomialNB(), param_grid={
    'alpha': np.arange(0.1, 10.1, 0.1),
    'fit_prior': [False, True]
}, cv=3).fit(data.data, data.target)
print(res.best_params_, res.best_score_)

{'alpha': 2.7, 'fit_prior': False} 0.9666666666666667
CPU times: user 716 ms, sys: 4 ms, total: 720 ms
Wall time: 719 ms


/home/tylorn/envs/python_data_science/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


# GaussianNB

В GaussianNB вероятность значений признака предполагается распределенной по Гауссу:

$$P(x_i \mid y) = \frac{1}{\sqrt{2\pi\sigma^2_y}} \exp\left(-\frac{(x_i - \mu_y)^2}{2\sigma^2_y}\right)$$

Параметры $\sigma_y$ and $\mu_y$ оцениваются с помощью метода максимального правдоподобия.

Добавим в сравнение GaussianNB

In [17]:
from sklearn.naive_bayes import GaussianNB

cross_val_score(GaussianNB(), data.data, data.target)

/home/tylorn/envs/python_data_science/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


array([0.92156863, 0.90196078, 0.97916667])

In [18]:
cross_val_score(GaussianNB(), data.data, data.target).mean()

/home/tylorn/envs/python_data_science/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


0.9342320261437909

# 20 newsgroups dataset

Проверим работу методов на другом наборе данных

In [19]:
from sklearn.datasets import fetch_20newsgroups_vectorized

data = fetch_20newsgroups_vectorized(subset='all', remove=('headers', 'footers', 'quotes'))

In [20]:
data.data

<18846x101631 sparse matrix of type '<class 'numpy.float64'>'
	with 1769365 stored elements in Compressed Sparse Row format>

In [21]:
cross_val_score(BernoulliNB(), data.data, data.target)

/home/tylorn/envs/python_data_science/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


array([0.46581876, 0.50334501, 0.46670914])

In [22]:
%%time
res = RandomizedSearchCV(BernoulliNB(), param_distributions={
    'binarize': scipy.stats.uniform(0, 1),
    'alpha': scipy.stats.uniform(0, 10),
    'fit_prior': [False, True]
}, cv=3, random_state=42).fit(data.data, data.target)
print(res.best_params_, res.best_score_)

{'alpha': 0.07066305219717406, 'binarize': 0.023062425041415757, 'fit_prior': False} 0.6818423007534755
CPU times: user 10.5 s, sys: 1.16 s, total: 11.7 s
Wall time: 11.7 s


In [23]:
cross_val_score(MultinomialNB(), data.data, data.target)

/home/tylorn/envs/python_data_science/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


array([0.55023847, 0.54858235, 0.51258363])

In [24]:
%%time
res = RandomizedSearchCV(MultinomialNB(), param_distributions={
    'alpha': scipy.stats.uniform(0.1, 10),
    'fit_prior': [False, True]
}, cv=3, random_state=42).fit(data.data, data.target)
print(res.best_params_, res.best_score_)

{'alpha': 0.10778765841014329, 'fit_prior': True} 0.6743075453677173
CPU times: user 6.76 s, sys: 936 ms, total: 7.69 s
Wall time: 7.69 s


In [25]:
cross_val_score(GaussianNB(), data.data, data.target)

/home/tylorn/envs/python_data_science/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/home/tylorn/envs/python_data_science/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:542: FutureWarning: From version 0.22, errors during fit will result in a cross validation score of NaN by default. Use error_score='raise' if you want an exception raised or error_score=np.nan to adopt the behavior from version 0.22.
  FutureWarning)


TypeError: A sparse matrix was passed, but dense data is required. Use X.toarray() to convert to a dense numpy array.

# Wine dataset

Может сложиться ощущение, что GaussianNB работает хуже, однако, когда признаки вещественные, это не так

In [26]:
from sklearn.datasets import load_wine

data = load_wine()

In [27]:
data.data

array([[1.423e+01, 1.710e+00, 2.430e+00, ..., 1.040e+00, 3.920e+00,
        1.065e+03],
       [1.320e+01, 1.780e+00, 2.140e+00, ..., 1.050e+00, 3.400e+00,
        1.050e+03],
       [1.316e+01, 2.360e+00, 2.670e+00, ..., 1.030e+00, 3.170e+00,
        1.185e+03],
       ...,
       [1.327e+01, 4.280e+00, 2.260e+00, ..., 5.900e-01, 1.560e+00,
        8.350e+02],
       [1.317e+01, 2.590e+00, 2.370e+00, ..., 6.000e-01, 1.620e+00,
        8.400e+02],
       [1.413e+01, 4.100e+00, 2.740e+00, ..., 6.100e-01, 1.600e+00,
        5.600e+02]])

In [28]:
cross_val_score(BernoulliNB(), data.data, data.target)

/home/tylorn/envs/python_data_science/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


array([0.4       , 0.4       , 0.39655172])

In [29]:
%%time
res = RandomizedSearchCV(BernoulliNB(), param_distributions={
    'binarize': scipy.stats.uniform(0, 1000),
    'alpha': scipy.stats.uniform(0, 10),
    'fit_prior': [False, True]
}, cv=3, random_state=42, n_iter=500).fit(data.data, data.target)
print(res.best_params_, res.best_score_)

{'alpha': 3.559726786512616, 'binarize': 757.8461104643691, 'fit_prior': False} 0.6966292134831461
CPU times: user 2.44 s, sys: 12 ms, total: 2.46 s
Wall time: 2.45 s


/home/tylorn/envs/python_data_science/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [30]:
cross_val_score(MultinomialNB(), data.data, data.target)

/home/tylorn/envs/python_data_science/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


array([0.71666667, 0.81666667, 0.96551724])

In [31]:
%%time
res = RandomizedSearchCV(MultinomialNB(), param_distributions={
    'alpha': scipy.stats.uniform(0, 10),
    'fit_prior': [False, True]
}, cv=3, random_state=42, n_iter=500).fit(data.data, data.target)
print(res.best_params_, res.best_score_)

{'alpha': 3.745401188473625, 'fit_prior': False} 0.8426966292134831
CPU times: user 2.04 s, sys: 0 ns, total: 2.04 s
Wall time: 2.04 s


/home/tylorn/envs/python_data_science/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [32]:
cross_val_score(GaussianNB(), data.data, data.target)

/home/tylorn/envs/python_data_science/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


array([0.95      , 0.96666667, 0.96551724])

In [33]:
cross_val_score(GaussianNB(), data.data, data.target).mean()

/home/tylorn/envs/python_data_science/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


0.960727969348659